In [1]:
import torch
import transformers

/home/huypn10/miniconda3/envs/adaptation/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from model.adapter.petl_roberta_modeling  import PetlRobertaForSequenceClassification
model = PetlRobertaForSequenceClassification.from_pretrained("/home/huypn10/adaptation/checkpoints/glue/sst2/20230202Feb/glue.sst2.am_adapter.ao_sequential.unfrz_ef_.ne10.warm0.06.wd0.1.seed4..m_0.2.beta2_0.7.s_0.1")

In [54]:
def _make_causal_mask(input_ids_shape: torch.Size, dtype: torch.dtype, past_key_values_length: int = 0):
    bsz, tgt_len = input_ids_shape
    mask = torch.full((tgt_len, tgt_len), torch.tensor(torch.finfo(dtype).min))
    mask_cond = torch.arange(mask.size(-1))
    mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 0)
    mask = mask.to(dtype)

    if past_key_values_length > 0:
        mask = torch.cat([torch.zeros(tgt_len, past_key_values_length, dtype=dtype), mask], dim=-1)
    return mask

In [5]:
def momentum_matrix(n, m, s, opt):   
    mask_matrix = torch.zeros([n,n], dtype=torch.float32, requires_grad=False)
    if opt == "origin" or opt == "test":
        for i in range(n):
            mask_matrix[i,i] = 1
            for j in range(1,i+1):
                mask_matrix[i,i-j] = mask_matrix[i,i-j+1] * m
                # mask_matrix[i,i-j] = (mask_matrix[i,i-j+1] - 0.1) if (mask_matrix[i,i-j+1] - 0.1) > 0 else 0|
    elif opt == "bidirectional":
        for i in range(n):
            mask_matrix[i,i] = 1
            for j in range(1,i+1):
                mask_matrix[i,i-j] = mask_matrix[i,i-j+1] * m
                # mask_matrix[i,i-j] = (mask_matrix[i,i-j+1] - 0.1) if (mask_matrix[i,i-j+1] - 0.1) > 0 else 0
            for j in range(i+1,n):
                mask_matrix[i,j] = mask_matrix[i,j-1] * m
    elif opt== "threshold" or opt == "adam":
        for i in range(n):
            mask_matrix[i,i] = 1
            for j in range(1,min(10,i+1)):
                mask_matrix[i,i-j] = mask_matrix[i,i-j+1] * m
    elif opt== "nesterov":
        for i in range(n):
            m=i/(i+3)
            mask_matrix[i,i] = 1
            for j in range(1,i+1):
                mask_matrix[i,i-j] = mask_matrix[i,i-j+1] * m
    else: return None

    if opt != "adam":
        mask_matrix = s*mask_matrix

    return mask_matrix

In [6]:
momentum_matrix(5,0.5,0.1,"bidirectional")

tensor([[0.1000, 0.0500, 0.0250, 0.0125, 0.0063],
        [0.0500, 0.1000, 0.0500, 0.0250, 0.0125],
        [0.0250, 0.0500, 0.1000, 0.0500, 0.0250],
        [0.0125, 0.0250, 0.0500, 0.1000, 0.0500],
        [0.0063, 0.0125, 0.0250, 0.0500, 0.1000]])